In [1]:
from datetime import datetime
import dill

from src.learner import pexp_mind
from src.core.match import Match

/opt/conda/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_pexp(size=9):
    mind = pexp_mind.PExpMind(size=size, init=False, channels=4)
    mind.load_net('../trained_models/9_4_4')
    
    def expanding_p(depth, p):
        import numpy as np
        return np.logical_or.reduce([
            np.logical_and(depth < 2, p > -6),
            np.logical_and(depth < 4, p > -4),
            np.logical_and(depth < 6, p > -4),
            np.logical_and(depth < np.inf, p > -3)
        ])

    def permissive_expansion(depth):
        import numpy as np
        if depth < 2:
            return np.inf
        if depth < 8:
            return 5
        return 3

    mind.define_policies(expanding_p, permissive_expansion, convergence_count=5,
                         alpha=0.2, q_exp_batch_size=size ** 2,
                         p_exp_batch_size=size ** 3, required_depth=6, max_iters=20)
    
    return mind

In [3]:
size = 9
players_bc = sc.broadcast({
    0: dill.dumps(make_pexp(size))
})

In [4]:
def play_game(player_index_1, player_index_2):
    model_1 = dill.loads(players_bc.value[player_index_1])
    
    # don't load twice if we don't need to
    if player_index_1 == player_index_2:
        model_2 = model_1
    else:
        model_2 = dill.loads(players_bc.value[player_index_2])
        
    match = Match(model_1, model_2, trivialize=False)
    return match.play()

now = datetime.now().strftime("%Y-%m-%d-%H-%M")
filepath = "gs://itd-aia-ds-dproc-staging/q-gomoku/games" + now
for i in range(1):
    num_games = 100
    sc.parallelize([(i, (0, 0)) for i in range(num_games)]).repartition(num_games) \
                    .partitionBy(num_games, lambda x : x) \
                    .map(lambda x : (play_game(x[1][0], x[1][1]), )).toDF(['gamestring']) \
                    .write.parquet(filepath, mode='overwrite')

In [13]:
!gsutil ls gs://itd-aia-ds-dproc-staging/q-gomoku/

gs://itd-aia-ds-dproc-staging/q-gomoku/
gs://itd-aia-ds-dproc-staging/q-gomoku/.gitignore
gs://itd-aia-ds-dproc-staging/q-gomoku/LICENSE
gs://itd-aia-ds-dproc-staging/q-gomoku/Makefile
gs://itd-aia-ds-dproc-staging/q-gomoku/README
gs://itd-aia-ds-dproc-staging/q-gomoku/__init__.py
gs://itd-aia-ds-dproc-staging/q-gomoku/environment.yml
gs://itd-aia-ds-dproc-staging/q-gomoku/fs_wait_train.py
gs://itd-aia-ds-dproc-staging/q-gomoku/.git/
gs://itd-aia-ds-dproc-staging/q-gomoku/.idea/
gs://itd-aia-ds-dproc-staging/q-gomoku/conf/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-00-00-00/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-00:00:00/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-06-37-51/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-06-43-09/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-16-05-49/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-21-30/
gs://itd-aia-ds-dproc-staging/q-gomoku/games_1/
gs://itd-aia-ds-dproc-staging/q-gomo

In [15]:
spark.read.parquet("gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-21-30/").collect()

[Row(gamestring='{"initial_state": "9.120000000120000000120000000120000000000000000000000000000000000000000000000000000.1", "moves": [[4, 0]], "winning_player": 1, "q_assessments": [[-0.02823619917035103, 1.0]]}')]